In [31]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import csv
# Pour le test statistique
import scipy.stats as stats
import math
import numpy as np

#cols = ['Period', 'Période pédagogique', 'Civilité', 'Nom et prénom',
#            'Orientation Bachelor', 'Orientation Master', 'Filière opt.',
#            'Mineur', 'Statut', 'Type', 'Ecole Echange', 'Sciper']

cols = ['Période académique', 'Période pédagogique', 'Civilité', 'Nom et prénom','Sciper','Mineur','Spécialisation','Statut']
ALL_DATA = pd.DataFrame(columns=cols)

In [32]:
def arrange_student(student_tags) : 
    
    student = {}
    
    if(len(student_tags)!=0) : 
        
        student = {}
    
        student['Civilité'] = student_tags[0].contents[0]
        student['Nom et prénom'] = student_tags[1].contents[0]
        student['Sciper'] = student_tags[10].contents[0]
        student['Statut'] = student_tags[7].contents[0]
        
        if(len(student_tags[4].contents)!=0) : 
            student['Spécialisation'] = student_tags[4].contents[0]
        
        if(len(student_tags[6].contents)!=0) : 
            student['Mineur'] = student_tags[6].contents[0]
            
        
    return student

In [33]:
#ACQUISITION DES FILTRES SOUS FORME DE DICTIONNAIRE : OK 

url1 = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247'
r1 = requests.get(url1)
soup1 = BeautifulSoup(r1.content,"lxml")

filters = {}

for filt in soup1.findAll("select"):
    
    filter_values = {}
        
    for option in filt.findAll("option"):
       
        if(option['value']!='null') : #juste pr éviter le premier element ki est vide
            filter_values[option.contents[0]] = option['value']
    
    filter_values[''] = 'null'
    filters[filt['name']] = filter_values

In [34]:
#URL avec lequel on va faire les requêtes et fichier csv qu'on va créer 

url2 = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'

#Paramètres qu'on va passer aux filtres
fixed_params = {}
fixed_params['ww_x_GPS'] = '-1'
fixed_params['ww_i_reportModel'] = '133685247'
fixed_params['ww_i_reportModelXsl'] = '133685270'
fixed_params['ww_x_UNITE_ACAD'] = filters['ww_x_UNITE_ACAD']['Informatique']

#Truc ou on va stocker tt les etudiants de master quelque soit leur niveau

master_students = []

for periode_pedagogique in ['Master semestre 1','Master semestre 2','Master semestre 3',
                            'Projet Master automne','Projet Master printemps'] :

    for periode_academique in filters['ww_x_PERIODE_ACAD'].keys() : 
        
        if(periode_academique != '') :
    
            parameters = fixed_params.copy()
            parameters['ww_x_PERIODE_PEDAGO'] = filters['ww_x_PERIODE_PEDAGO'][periode_pedagogique]
            parameters['ww_x_PERIODE_ACAD'] = filters['ww_x_PERIODE_ACAD'][periode_academique]        
            
            req = requests.get(url2,params=parameters)
            soup = BeautifulSoup(req.content,"lxml")
            
            
            for row in soup.find('table').contents[2:] : #on commence 2ème ligne pasque les premières c'est des headers
       
                s = arrange_student(row.findAll('td'))
                s['Période académique'] = periode_academique
                s['Période pédagogique'] = periode_pedagogique
                master_students.append(s)

In [35]:
for student in master_students : 
    ALL_DATA = ALL_DATA.append(pd.Series(student), ignore_index=True)
ALL_DATA=ALL_DATA = ALL_DATA.drop_duplicates()

In [37]:
ALL_DATA=ALL_DATA.sort_values('Nom et prénom')
NEW_ALL_DATA=ALL_DATA.set_index(['Nom et prénom'])
NEW_ALL_DATA

,Période académique,Période pédagogique,Civilité,Sciper,Mineur,Spécialisation,Statut
Nom et prénom,,,,,,,
Abbadi Hajar,2013-2014,Master semestre 1,Madame,238067,NaN,NaN,Présent
Abbadi Hajar,2014-2015,Master semestre 3,Madame,238067,"Mineur en Management, technologie et entrepren...",NaN,Présent
Abbadi Hajar,2013-2014,Master semestre 2,Madame,238067,"Mineur en Management, technologie et entrepren...",NaN,Présent
Abelenda Diego,2010-2011,Master semestre 2,Monsieur,170646,NaN,"Signals, Images and Interfaces",Présent
Abelenda Diego,2010-2011,Master semestre 1,Monsieur,170646,NaN,"Signals, Images and Interfaces",Présent
Abelenda Diego,2011-2012,Master semestre 2,Monsieur,170646,NaN,NaN,Présent
Abelenda Diego,2011-2012,Master semestre 3,Monsieur,170646,NaN,NaN,Présent
Abi Akar Nora,2015-2016,Master semestre 2,Madame,251253,NaN,NaN,Présent
Abi Akar Nora,2015-2016,Master semestre 1,Madame,251253,NaN,NaN,Présent


In [38]:
cols_result = ['Name','Sex','First year of master', 'Last year of master', 'Time_Master (months)','Spécialisation']
result = pd.DataFrame(columns=cols_result)

n=0
for name in NEW_ALL_DATA.index.drop_duplicates() :
    if(NEW_ALL_DATA.loc[name].shape[0]==7):
        n=n+1
    else:
        
        temp_frame = NEW_ALL_DATA.loc[name]
        # max_year = str(max(temp_frame['Période académique'])).split("-")[1]
        # min_year = str(min(temp_frame['Période académique'])).split("-")[0]
        t = temp_frame.shape[0]*6
        temp = {'Name' : name,'Sex' : str(NEW_ALL_DATA.loc[name]['Civilité'].values[0]) ,'First year of master' :temp_frame['Période académique'].min(),
                'Last year of master' :temp_frame['Période académique'].max() ,'Time_Master (months)' : t,
                'Spécialisation': str(NEW_ALL_DATA.loc[name]['Spécialisation'].values[0])}
        result = result.append(temp, ignore_index=True)
print(n)

191


In [39]:
result

,Name,Sex,First year of master,Last year of master,Time_Master (months),Spécialisation
0,Abbadi Hajar,Madame,2013-2014,2014-2015,18.0,nan
1,Abelenda Diego,Monsieur,2010-2011,2011-2012,24.0,"Signals, Images and Interfaces"
2,Abi Akar Nora,Madame,2015-2016,2016-2017,18.0,nan
3,Aeberhard François-Xavier,Monsieur,2007-2008,2009-2010,36.0,Internet computing
4,Aeby Prisca,Madame,2015-2016,2016-2017,18.0,Internet Information Systems
5,Agarwal Megha,Madame,2007-2008,2008-2009,18.0,nan
6,Ahmed Fayzan,Monsieur,2013-2014,2014-2015,18.0,nan
7,Alnasser Riyadh Abdulaziz M,Monsieur,2015-2016,2016-2017,18.0,nan
8,Alsudairy Sara,Madame,2010-2011,2010-2011,12.0,nan
9,Amiguet Jérôme,Monsieur,2014-2015,2016-2017,30.0,nan


In [65]:
result.mean()[0]

19.977099236641223

In [41]:
#Temps moyen passé par sexe : 

result_by_sex = result.groupby('Sex')
result_by_sex.mean()


,Time_Master (months)
Sex,
Madame,20.000000
Monsieur,19.973568


Computing arrays for Men/Women Time bachelor values.

In [55]:
results_men = result[result.Sex=='Monsieur']
results_men.head()

,Name,Sex,First year of master,Last year of master,Time_Master (months),Spécialisation
1,Abelenda Diego,Monsieur,2010-2011,2011-2012,24.0,"Signals, Images and Interfaces"
3,Aeberhard François-Xavier,Monsieur,2007-2008,2009-2010,36.0,Internet computing
6,Ahmed Fayzan,Monsieur,2013-2014,2014-2015,18.0,nan
7,Alnasser Riyadh Abdulaziz M,Monsieur,2015-2016,2016-2017,18.0,nan
9,Amiguet Jérôme,Monsieur,2014-2015,2016-2017,30.0,nan


In [48]:
men = np.array(results_men['Time_Master (months)'])

In [49]:
results_women = result[result.Sex=='Madame']
women = np.array(results_women['Time_Master (months)'])

Doing a Two Sample T-test assuming that both samples (men and women) do not have the same variance. In effect, we don't have information about that.

The null hypothesis is that the mean of both groups are the same. We choose a significant level of 0.05

In [45]:
stats.ttest_ind(a= men,
                b= women,
                equal_var=False) 

Ttest_indResult(statistic=-0.042945428864118659, pvalue=0.96580682435779752)

In [47]:
result.groupby('Spécialisation').mean()

,Time_Master (months)
Spécialisation,
Biocomputing,21.600000
Computer Engineering - SP,21.750000
Data Analytics,17.000000
Foundations of Software,23.076923
Information Security - SP,30.000000
Internet Information Systems,18.000000
Internet computing,22.875000
Service science,24.000000
"Signals, Images and Interfaces",23.454545


In [67]:
for spe in result['Spécialisation'].drop_duplicates():
    results_spe = result[result.Spécialisation==spe]
    spe_array = np.array(results_spe['Time_Master (months)'])
    print('For the specialisation : ', spe, 'here is the result of the test')
    print(stats.ttest_1samp(a= spe_array,               
                 popmean= result.mean()[0]))

For the specialisation :  nan here is the result of the test
Ttest_1sampResult(statistic=-3.8395395855081169, pvalue=0.00013663777119937435)
For the specialisation :  Signals, Images and Interfaces here is the result of the test
Ttest_1sampResult(statistic=2.2813377137674027, pvalue=0.033067777850080485)
For the specialisation :  Internet computing here is the result of the test
Ttest_1sampResult(statistic=4.3423230956270435, pvalue=4.1425696399630932e-05)
For the specialisation :  Internet Information Systems here is the result of the test
Ttest_1sampResult(statistic=nan, pvalue=nan)
For the specialisation :  Software Systems here is the result of the test
Ttest_1sampResult(statistic=1.108814499579786, pvalue=0.29116420317869074)
For the specialisation :  Foundations of Software here is the result of the test
Ttest_1sampResult(statistic=4.1668346300656784, pvalue=0.00011948829252962149)
For the specialisation :  Biocomputing here is the result of the test
Ttest_1sampResult(statistic=0

/Users/Gusto/anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


ON ESSAIE De faire comme si tout le monde avait un projet

In [ ]:

result_copy=result.copy() 
for ind, name in enumerate(ALL_DATA['Période pédagogique']):
        if(name.startswith('Projet')):
            print('true')
            print(ind)
            print(result_copy['Time_Master (months)'])
            
            #result['Time_Master (months)'][ind] -= 6
            #print(result['Time_Master (months)'][ind-1])
            
#ALL_DATA[ALL_DATA['Période pédagogique'].startswith('Projet')]